# 高级自动微分
重点介绍 tf.GradientTape API 更深入、更不常见的功能。

In [1]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)


## 控制梯度记录
在自动微分指南中，您已了解构建梯度计算时如何控制条带监视变量和张量。<br><br>

条带还具有操作记录的方法。<br><br>

如果您希望停止记录梯度，可以使用 GradientTape.stop_recording() 暂时挂起记录。<br><br>

---

In [4]:
x=tf.Variable(2.0)
y=tf.Variable(3.0)
with tf.GradientTape() as t:
    x_sq=x*x
    with t.stop_recording() :
        y_sq=y*y
    z=x_sq+y_sq
grad = t.gradient(z,{'x':x,'y':y})
print('dz_dx:',grad['x'].numpy())
print('dz_dy:',grad['y'])

dz_dx: 4.0
dz_dy: None


如果您希望完全重新开始，请使用 reset()。通常，直接退出梯度带块并重新开始比较易于读取，但在退出梯度带块有困难或不可行时，可以使用 reset。

In [5]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
  y_sq = y * y
  if reset:
    # Throw out all the tape recorded so far
    t.reset()
  z = x * x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])


dz/dx: tf.Tensor(4.0, shape=(), dtype=float32)
dz/dy: None


## 停止梯度
与上面的全局条带控制相比，tf.stop_gradient 函数更加精确。
<br><br>
它可以用来**阻止梯度沿着特定路径流动，而不需要访问条带本身**

In [7]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  y_sq = y**2
  z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])


dz/dx: tf.Tensor(4.0, shape=(), dtype=float32)
dz/dy: None


## 自定义梯度

    1. 正在编写的新运算没有定义的梯度。
    2. 默认计算在数值上不稳定。
    3. 您希望从前向传递缓存开销大的计算。
    4. 您想修改一个值（例如使用：tf.clip_by_value、tf.math.round）而不修改梯度。
    
    对于编写新运算，您可以使用 tf.RegisterGradient 自行设置。请参阅其页面了解详细信息。（注意，梯度注册为全局，需谨慎更改。）<br><br>
    对于后三种情况，可以使用 tf.custom_gradient。
